In [49]:
import os, re 
import numpy as np
import tensorflow as tf

# 파일을 읽기모드로 열고
# 라인 단위로 끊어서 list 형태로 읽어옵니다.
file_path = os.getenv('HOME') + '/aiffel/lyricist/data/shakespeare.txt'
with open(file_path, "r") as f:
    raw_corpus = f.read().splitlines()

# 앞에서부터 10라인만 화면에 출력해 볼까요?
print(raw_corpus[:9])

['First Citizen:', 'Before we proceed any further, hear me speak.', '', 'All:', 'Speak, speak.', '', 'First Citizen:', 'You are all resolved rather to die than to famish?', '']


In [50]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뜁니다.

    if idx > 9: break   # 일단 문장 10개만 확인해 볼 겁니다.
        
    print(sentence)

Before we proceed any further, hear me speak.
Speak, speak.
You are all resolved rather to die than to famish?


In [51]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

# 이 문장이 어떻게 필터링되는지 확인해 보세요.
print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


In [63]:
corpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    if len(sentence) > 30: continue
    
    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인해보죠
corpus[:10]

['<start> speak , speak . <end>',
 '<start> resolved . resolved . <end>',
 '<start> we know t , we know t . <end>',
 '<start> is t a verdict ? <end>',
 '<start> one word , good citizens . <end>',
 '<start> come , come . <end>',
 '<start> soft ! who comes here ? <end>',
 '<start> the people . <end>',
 '<start> have strong arms too . <end>',
 '<start> will you undo yourselves ? <end>']

In [64]:
def tokenize(corpus):
    # 7000단어를 기억할 수 있는 tokenizer를 만들겁니다
    # 우리는 이미 문장을 정제했으니 filters가 필요없어요
    # 7000단어에 포함되지 못한 단어는 '<unk>'로 바꿀거에요
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=18000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2   85    5 ...    0    0    0]
 [   2  554    4 ...    0    0    0]
 [   2   45   70 ...    0    0    0]
 ...
 [   2   13   99 ...    0    0    0]
 [   2   92 1115 ...    0    0    0]
 [   2 2507   40 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7fbb5c2a1160>


In [65]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : .
5 : ,
6 : ?
7 : i
8 : !
9 : you
10 : the


In [66]:
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[ 2 85  5 85  4  3  0  0  0  0  0  0  0]
[85  5 85  4  3  0  0  0  0  0  0  0  0]


In [67]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
# 데이터셋에 대해서는 아래 문서를 참고하세요
# 자세히 알아둘수록 도움이 많이 되는 중요한 문서입니다
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 13), (256, 13)), types: (tf.int32, tf.int32)>

In [68]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [58]:
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)

<tf.Tensor: shape=(256, 9, 18001), dtype=float32, numpy=
array([[[ 6.84684128e-05,  1.86212870e-04,  1.74469125e-04, ...,
          1.45813057e-04, -9.73324422e-05,  1.12704525e-04],
        [ 7.02266625e-05,  4.08741733e-04,  1.57959454e-04, ...,
          2.46080075e-04, -1.34927497e-04,  2.51455291e-04],
        [-7.03496562e-06,  7.98054389e-04,  4.39115713e-04, ...,
          4.45217709e-04, -1.67473030e-04,  2.92879384e-04],
        ...,
        [-2.08977493e-04,  1.24841882e-03,  1.42491853e-03, ...,
          3.34338314e-04,  1.04784640e-03,  2.46391702e-03],
        [-2.99755920e-04,  1.31006364e-03,  1.64817006e-03, ...,
          2.46683107e-04,  1.30798435e-03,  2.93986010e-03],
        [-4.00110759e-04,  1.37711735e-03,  1.85169652e-03, ...,
          1.58774303e-04,  1.51947245e-03,  3.33466171e-03]],

       [[ 6.84684128e-05,  1.86212870e-04,  1.74469125e-04, ...,
          1.45813057e-04, -9.73324422e-05,  1.12704525e-04],
        [ 1.23662554e-04,  1.52607332e-04,  2.

In [59]:
model.summary()

Model: "text_generator_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      multiple                  4608256   
_________________________________________________________________
lstm_12 (LSTM)               multiple                  5246976   
_________________________________________________________________
lstm_13 (LSTM)               multiple                  8392704   
_________________________________________________________________
dense_6 (Dense)              multiple                  18451025  
Total params: 36,698,961
Trainable params: 36,698,961
Non-trainable params: 0
_________________________________________________________________


In [69]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=30)

Epoch 1/30
15/15 [==============================] - 7s 202ms/step - loss: 5.6080
Epoch 2/30
15/15 [==============================] - 3s 201ms/step - loss: 3.4662
Epoch 3/30
15/15 [==============================] - 3s 203ms/step - loss: 3.0859
Epoch 4/30
15/15 [==============================] - 3s 203ms/step - loss: 2.7646
Epoch 5/30
15/15 [==============================] - 3s 203ms/step - loss: 2.6125
Epoch 6/30
15/15 [==============================] - 3s 200ms/step - loss: 2.4888
Epoch 7/30
15/15 [==============================] - 3s 200ms/step - loss: 2.3844
Epoch 8/30
15/15 [==============================] - 3s 199ms/step - loss: 2.3100
Epoch 9/30
15/15 [==============================] - 3s 198ms/step - loss: 2.2706
Epoch 10/30
15/15 [==============================] - 3s 198ms/step - loss: 2.2330
Epoch 11/30
15/15 [==============================] - 3s 198ms/step - loss: 2.2203
Epoch 12/30
15/15 [==============================] - 3s 198ms/step - loss: 2.1999
Epoch 13/30
15/15 [======

In [71]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [72]:
generate_text(model, tokenizer, init_sentence="<start> he")

'<start> he is not , my lord . <end> '

In [78]:
import glob
import os
from sklearn.model_selection import train_test_split

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]


In [80]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (3171, 13)
Target Train: (3171, 13)


Now I've heard there was a secret chord
That David played, and it pleased the Lord
But you don't really care for music, do you?
It goes like this
The fourth, the fifth
The minor fall, the major lift
The baffled king composing Hallelujah Hallelujah
Hallelujah
Hallelujah
Hallelujah Your faith was strong but you needed proof


In [83]:
corpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    if len(sentence) > 30: continue
    
    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인해보죠
corpus[:10]

['<start> it goes like this <end>',
 '<start> the fourth , the fifth <end>',
 '<start> the minor fall , the major lift <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> she tied you <end>',
 '<start> to a kitchen chair <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> i don t even know the name <end>']

In [87]:
tensor, tokenizer = tokenize(corpus)

[[    2    11   262 ...     0     0     0]
 [    2     7  3145 ...     0     0     0]
 [    2     7  3997 ...     0     0     0]
 ...
 [    2     4    48 ...     0     0     0]
 [    2     6 14042 ...     0     0     0]
 [    2     6    43 ...     0     0     0]] <keras_preprocessing.text.Tokenizer object at 0x7fbc5c9cab50>


In [85]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : i
5 : ,
6 : you
7 : the
8 : and
9 : a
10 : to


In [86]:
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[  2  11 262  38  49   3   0   0   0   0   0   0   0   0   0   0   0   0
   0]
[ 11 262  38  49   3   0   0   0   0   0   0   0   0   0   0   0   0   0
   0]


In [88]:
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input,tgt_input,
                                                          test_size=0.2, 
                                                         random_state=7)

In [89]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
# 데이터셋에 대해서는 아래 문서를 참고하세요
# 자세히 알아둘수록 도움이 많이 되는 중요한 문서입니다
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 19), (256, 19)), types: (tf.int32, tf.int32)>

In [90]:
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)

<tf.Tensor: shape=(256, 19, 18001), dtype=float32, numpy=
array([[[  1.3036066 ,  -8.853216  ,  -9.087038  , ...,  -9.108093  ,
          -9.155725  ,  -8.931289  ],
        [  3.013279  , -13.087436  , -13.172214  , ..., -13.34543   ,
         -13.271196  , -13.118526  ],
        [ 13.912555  , -21.731083  , -21.962103  , ..., -22.054375  ,
         -22.058767  , -22.002867  ],
        ...,
        [ 16.668236  , -23.727633  , -23.90971   , ..., -24.120771  ,
         -23.935247  , -24.091442  ],
        [ 16.668932  , -23.729017  , -23.91083   , ..., -24.122227  ,
         -23.936693  , -24.093155  ],
        [ 16.669516  , -23.730263  , -23.911837  , ..., -24.123562  ,
         -23.937983  , -24.094748  ]],

       [[  1.3036066 ,  -8.853216  ,  -9.087038  , ...,  -9.108093  ,
          -9.155725  ,  -8.931289  ],
        [ -1.1286628 ,  -8.972323  ,  -9.068763  , ...,  -9.055562  ,
          -9.043739  ,  -8.944218  ],
        [ -0.82287496, -10.118118  , -10.241926  , ..., -10.194

In [91]:
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [ ]:
model(enc_train)

In [95]:
model.summary()

Model: "text_generator_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      multiple                  4608256   
_________________________________________________________________
lstm_16 (LSTM)               multiple                  5246976   
_________________________________________________________________
lstm_17 (LSTM)               multiple                  0 (unused)
_________________________________________________________________
dense_8 (Dense)              multiple                  0 (unused)
Total params: 9,855,232
Trainable params: 9,855,232
Non-trainable params: 0
_________________________________________________________________


In [96]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=30)

Epoch 1/30
227/227 [==============================] - 71s 300ms/step - loss: 2.1396
Epoch 2/30
227/227 [==============================] - 68s 299ms/step - loss: 1.6419
Epoch 3/30
227/227 [==============================] - 68s 297ms/step - loss: 1.5426
Epoch 4/30
227/227 [==============================] - 68s 298ms/step - loss: 1.4662
Epoch 5/30
227/227 [==============================] - 67s 296ms/step - loss: 1.4040
Epoch 6/30
227/227 [==============================] - 67s 296ms/step - loss: 1.3497
Epoch 7/30
227/227 [==============================] - 68s 298ms/step - loss: 1.2996
Epoch 8/30
227/227 [==============================] - 68s 299ms/step - loss: 1.2533
Epoch 9/30
227/227 [==============================] - 68s 299ms/step - loss: 1.2092
Epoch 10/30
227/227 [==============================] - 68s 299ms/step - loss: 1.1682
Epoch 11/30
227/227 [==============================] - 68s 298ms/step - loss: 1.1290
Epoch 12/30
227/227 [==============================] - 68s 298ms/step - lo

In [104]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=150)

'<start> i love you <end> '